# Finetuning SGD Notebook
------------------
This notebook contains the code used to train the models with the SGD optimizer. 

It functions primarily on Google Colab, but can be adapted to work on local hardware.

It trains only one type of model and searches through hyperparameters with a Grid Search algorithm.

For each model type, minimal adjustments need to be made before running the code again.

In [ ]:
# This script mounts Google Drive in a Google Colab environment.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Set the current working directory to a specific path in Google Drive.
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


## Global Imports

In [ ]:
# Import necessary libraries for the project.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from Functions.implementations import * # Import custom implementations, these contain the model definitions and training functions.

# Training with SGD optimizer :

## Load Data

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
train_loader,valid_loader, test_loader = get_data_loaders()

Train and evaluate a simple model to test if everything works.

## SGD :

In [ ]:
import os

#only for google Colab :

# Set a path in your Google Drive
csv_path = "/content/drive/MyDrive/OptiML/repo/OptML-project_SGD_DenseNet_Transform.csv"
save_path = "/content/drive/MyDrive/OptiML/repo/OptML-project/Results/SGD"
# For local machine, set the path to your desired location

# Set the number of epochs and evaluation interval
epochs = 100
eval_interval = 10 # Interval for evaluation and saving results

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "beta_1","beta_2"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set seed for reproducibility through custom function as done throughtout the project

In [ ]:
from itertools import product

# Define the device to use for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get data loaders for training, validation, and testing
train_loader,valid_loader, test_loader = get_data_loaders(batch_size=128)

# Set up a dictionary to store results
results_grid = {}

# Define the hyperparameters for the grid search
# Chosen hyperparameters over which to perform the grid search
# learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
# momentums = [0.0, 0.6, 0.9, 0.99]

learning_rates = [1e-3]
momentums = [0.99]

# Iterate over all combinations of lr and momentum
for lr, momentum in product(learning_rates, momentums):
    scores, model = train_and_return_evaluation_SGD(
        get_densenet121, # Model type that needs to be trained, needs to be changed for different model types. Options are defined in the implementations.py file.
        lr=lr,
        momentum=momentum,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, momentum)] = scores # Store the scores for each combination of hyperparameters

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]

    row = [lr, momentum] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)

    # Save the model
    torch.save(model.state_dict(), save_path + f"/DenseNet_Transform_lr_{lr}_momentum_{momentum}.pth")


🔧 Training with SGD: lr=0.001, momentum=0.99
Epoch 10 | Acc=0.8162 | Recall=0.8157 | F1=0.8154
Epoch 20 | Acc=0.8638 | Recall=0.8639 | F1=0.8642
Epoch 30 | Acc=0.8834 | Recall=0.8831 | F1=0.8834
Epoch 40 | Acc=0.8872 | Recall=0.8876 | F1=0.8878
Epoch 50 | Acc=0.8894 | Recall=0.8895 | F1=0.8896
Epoch 60 | Acc=0.9026 | Recall=0.9025 | F1=0.9022
Epoch 70 | Acc=0.9042 | Recall=0.9041 | F1=0.9040
Epoch 80 | Acc=0.9094 | Recall=0.9095 | F1=0.9091
Epoch 90 | Acc=0.9064 | Recall=0.9068 | F1=0.9066
Epoch 100 | Acc=0.9068 | Recall=0.9070 | F1=0.9069

Test Set Evaluation: Acc=0.9065 | Recall=0.9065 | F1=0.9062
